In [1]:
import pandas as pd
import json

In [2]:
CSV_NAME = 'first_quarter_cleaned.csv'
VOCAB_FILE = 'vocab_dict.json'

In [3]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU available")

True
NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [4]:
df = pd.read_csv(CSV_NAME)
df.head()

,Unnamed: 0,Event,White,Black,Result,UTCDate,UTCTime,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,AN,AN_tok,AN_tok_int
0,0,Classical,eisaaaa,HAMID449,1-0,2016.06.30,22:00:01,1901,1896,11.0,-11.0,D10,Slav Defense,300+5,Time forfeit,1. d4 d5 2. c4 c6 3. e3 a6 4. Nf3 e5 5. cxd5 e...,"['1.', 'd4', 'd5', '2.', 'c4', 'c6', '3.', 'e3...","[1, 5271, 5272, 124, 5231, 5233, 237, 5311, 51..."
1,1,Blitz,go4jas,Sergei1973,0-1,2016.06.30,22:00:01,1641,1627,-11.0,12.0,C20,King's Pawn Opening: 2.b3,300+0,Normal,1. e4 e5 2. b3 Nf6 3. Bb2 Nc6 4. Nf3 d6 5. d3 ...,"['1.', 'e4', 'e5', '2.', 'b3', 'Nf6', '3.', 'B...","[1, 5312, 5313, 124, 5188, 1346, 237, 419, 113..."
2,2,Blitz tournament,Evangelistaizac,kafune,1-0,2016.06.30,22:00:02,1647,1688,13.0,-13.0,B01,Scandinavian Defense: Mieses-Kotroc Variation,180+0,Time forfeit,1. e4 d5 2. exd5 Qxd5 3. Nf3 Bg4 4. Be2 Nf6 5....,"['1.', 'e4', 'd5', '2.', 'exd5', 'Qxd5', '3.',...","[1, 5312, 5272, 124, 5327, 3302, 237, 1343, 49..."
3,3,Correspondence,Jvayne,Wsjvayne,1-0,2016.06.30,22:00:02,1706,1317,27.0,-25.0,A00,Van't Kruijs Opening,-,Normal,1. e3 Nf6 2. Bc4 d6 3. e4 e6 4. Nf3 Nxe4 5. Nd...,"['1.', 'e3', 'Nf6', '2.', 'Bc4', 'd6', '3.', '...","[1, 5311, 1346, 124, 436, 5273, 237, 5312, 531..."
4,4,Blitz tournament,kyoday,BrettDale,0-1,2016.06.30,22:00:02,1945,1900,-14.0,13.0,B90,"Sicilian Defense: Najdorf, Lipnitsky Attack",180+0,Time forfeit,1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. N...,"['1.', 'e4', 'c5', '2.', 'Nf3', 'd6', '3.', 'd...","[1, 5312, 5232, 124, 1343, 5273, 237, 5271, 52..."


In [5]:
with open(VOCAB_FILE, "r") as f:
    vocab_dict = json.load(f)
list(vocab_dict.items())[:10]

[('<PAD>', 0),
 ('1.', 1),
 ('1...', 2),
 ('10.', 3),
 ('10...', 4),
 ('100.', 5),
 ('100...', 6),
 ('101.', 7),
 ('102.', 8),
 ('103.', 9)]

In [8]:
result_keys = {'1-0': 0, '0-1': 1, '1/2-1/2': 2, '*': 3}

def get_encoded_result(string):
    return result_keys[string]

res = df.head()['Result'].apply(get_encoded_result)
res

0    0
1    1
2    0
3    0
4    1
Name: Result, dtype: int64

In [9]:
df['Result_enc'] = df['Result'].apply(get_encoded_result)
df['Result_enc'].value_counts()

Result_enc
0    778004
1    725808
2     59629
3       605
Name: count, dtype: int64

In [10]:
lengths = df['AN_tok_int'].apply(len)
lengths.describe()

count    1.564046e+06
mean     5.620655e+02
std      2.589149e+02
min      9.000000e+00
25%      3.840000e+02
50%      5.350000e+02
75%      7.220000e+02
max      4.469000e+03
Name: AN_tok_int, dtype: float64

In [11]:
def decode_array_strings(string):
    clipped = string[1:-1]
    divided = clipped.split(', ')
    return [int(x) for x in divided]

df['AN_tok_int'] = df['AN_tok_int'].apply(decode_array_strings)
df['AN_tok_int']

0          [1, 5271, 5272, 124, 5231, 5233, 237, 5311, 51...
1          [1, 5312, 5313, 124, 5188, 1346, 237, 419, 113...
2          [1, 5312, 5272, 124, 5327, 3302, 237, 1343, 49...
3          [1, 5311, 1346, 124, 436, 5273, 237, 5312, 531...
4          [1, 5312, 5232, 124, 1343, 5273, 237, 5271, 52...
                                 ...                        
1564041    [1, 5271, 5272, 124, 1343, 1131, 237, 5311, 53...
1564042    [1, 5312, 5232, 124, 1343, 5163, 237, 436, 531...
1564043    [1, 5188, 2, 5314, 124, 419, 125, 5273, 237, 5...
1564044    [1, 5271, 5272, 124, 5231, 5233, 237, 1128, 13...
1564045    [1, 5271, 1346, 124, 5231, 5398, 237, 1128, 52...
Name: AN_tok_int, Length: 1564046, dtype: object

In [12]:
cleaned = df[(df['Result_enc'] != 3) & (df['Result_enc'] != 2)]

In [13]:
import numpy as np
clean_X = cleaned['AN_tok_int']
# clean_X = [np.array(x) for x in clean_X]
clean_X[:2]

0    [1, 5271, 5272, 124, 5231, 5233, 237, 5311, 51...
1    [1, 5312, 5313, 124, 5188, 1346, 237, 419, 113...
Name: AN_tok_int, dtype: object

In [14]:
clean_Y = cleaned['Result_enc']
clean_Y

0          0
1          1
2          0
3          0
4          1
          ..
1564041    0
1564042    1
1564043    0
1564044    1
1564045    1
Name: Result_enc, Length: 1503812, dtype: int64

In [15]:
cleaned['AN_tok_int'].apply(len).describe()

count    1.503812e+06
mean     1.025083e+02
std      4.637165e+01
min      2.000000e+00
25%      7.100000e+01
50%      9.800000e+01
75%      1.310000e+02
max      8.940000e+02
Name: AN_tok_int, dtype: float64

In [30]:
print(cleaned['Result_enc'].value_counts())

Result_enc
0    778004
1    725808
Name: count, dtype: int64


In [16]:
max_len = 150  # Set your desired length

In [17]:
def pad_or_clip(lst, desired_length):
    return (lst + [0] * desired_length)[:desired_length]
padded_X = [pad_or_clip(x, max_len) for x in clean_X]

In [18]:
padded_X[:2]

[[1,
  5271,
  5272,
  124,
  5231,
  5233,
  237,
  5311,
  5163,
  259,
  1343,
  5313,
  281,
  5258,
  5312,
  303,
  1273,
  5258,
  325,
  2240,
  450,
  347,
  1538,
  1538,
  369,
  1128,
  1346,
  3,
  2371,
  464,
  16,
  1536,
  2241,
  28,
  1128,
  1572,
  40,
  459,
  5190,
  52,
  1572,
  4046,
  64,
  445,
  2502,
  76,
  4031,
  2637,
  88,
  2496,
  2770,
  100,
  1070,
  449,
  112,
  5160,
  1075,
  126,
  2500,
  1353,
  138,
  407,
  4285,
  150,
  2377,
  3158,
  162,
  5436,
  1129,
  174,
  542,
  5217,
  186,
  3302,
  4723,
  198,
  3309,
  4419,
  210,
  435,
  4555,
  222,
  2898,
  4554,
  234,
  1198,
  4689,
  239,
  3029,
  4556,
  241,
  1272,
  4825,
  243,
  2638,
  4691,
  245,
  1537,
  5120,
  247,
  2502,
  4825,
  249,
  3154,
  4556,
  251,
  5272,
  5356,
  253,
  5273,
  4420,
  255,
  4716,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [19]:
X_tensor = torch.tensor(padded_X)
X_tensor.shape

torch.Size([1503812, 150])

In [20]:
Y_tensor = torch.tensor(list(clean_Y))
Y_tensor.shape

torch.Size([1503812])

In [21]:
# Save the dataset containing the full moves
full_x_name = 'x_full.pt'
full_y_name = 'y_full.pt'

torch.save(X_tensor, full_x_name)
torch.save(Y_tensor, full_y_name)

In [22]:
# Now process the dataset to remove the last TAIL moves

# Parameters
TAIL_MOVES = 5
TOKS_PER_MOVE = 3
MIN_MOVES = 15

TAIL = TAIL_MOVES * TOKS_PER_MOVE # predict ten moves 
MIN_TOKS = MIN_MOVES * TOKS_PER_MOVE # Set a minimum length for game sequences



In [23]:
# Remove all games that have fewer than MIN_TOKS move tokens
df_min = cleaned[cleaned['AN_tok_int'].apply(len) >= MIN_TOKS]
df_min['AN_tok_int'].apply(len).describe()

count    1.362676e+06
mean     1.101517e+02
std      4.165687e+01
min      4.500000e+01
25%      8.000000e+01
50%      1.040000e+02
75%      1.350000e+02
max      8.940000e+02
Name: AN_tok_int, dtype: float64

In [24]:
# Remove the TAIL tokens from these games
def remove_tail(seq):
    return seq[:-TAIL]

df_min['AN_tok_trimmed'] = df_min['AN_tok_int'].apply(remove_tail)
df_min['AN_tok_trimmed'].apply(len).describe()

C:\Users\timec\AppData\Local\Temp\ipykernel_26996\2629193374.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_min['AN_tok_trimmed'] = df_min['AN_tok_int'].apply(remove_tail)


count    1.362676e+06
mean     9.515169e+01
std      4.165687e+01
min      3.000000e+01
25%      6.500000e+01
50%      8.900000e+01
75%      1.200000e+02
max      8.790000e+02
Name: AN_tok_trimmed, dtype: float64

In [25]:
# Now pad/trim these sequences and save the resulting tensor
import numpy as np
X_np = df_min['AN_tok_trimmed']
Y_np = df_min['Result_enc']

X_np.shape, Y_np.shape

((1362676,), (1362676,))

In [26]:
max_len = 150 # Arbitrary parameter
def pad_or_clip(lst, desired_length):
    return (lst + [0] * desired_length)[:desired_length]
X_np_pad = [pad_or_clip(x, max_len) for x in X_np]
X_np_pad[:2]

[[1,
  5271,
  5272,
  124,
  5231,
  5233,
  237,
  5311,
  5163,
  259,
  1343,
  5313,
  281,
  5258,
  5312,
  303,
  1273,
  5258,
  325,
  2240,
  450,
  347,
  1538,
  1538,
  369,
  1128,
  1346,
  3,
  2371,
  464,
  16,
  1536,
  2241,
  28,
  1128,
  1572,
  40,
  459,
  5190,
  52,
  1572,
  4046,
  64,
  445,
  2502,
  76,
  4031,
  2637,
  88,
  2496,
  2770,
  100,
  1070,
  449,
  112,
  5160,
  1075,
  126,
  2500,
  1353,
  138,
  407,
  4285,
  150,
  2377,
  3158,
  162,
  5436,
  1129,
  174,
  542,
  5217,
  186,
  3302,
  4723,
  198,
  3309,
  4419,
  210,
  435,
  4555,
  222,
  2898,
  4554,
  234,
  1198,
  4689,
  239,
  3029,
  4556,
  241,
  1272,
  4825,
  243,
  2638,
  4691,
  245,
  1537,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  

In [27]:
X_trim = torch.tensor(X_np_pad)
X_trim.shape

torch.Size([1362676, 150])

In [28]:
Y_trim = torch.tensor(list(Y_np))
Y_trim.shape

torch.Size([1362676])

In [29]:
# Save the dataset containing the full moves
trim_x_name = f"x_trim_{TAIL_MOVES}.pt"
trim_y_name = f'y_trim_{TAIL_MOVES}.pt'

torch.save(X_trim, trim_x_name)
torch.save(Y_trim, trim_y_name)

In [31]:
print(cleaned['AN_tok_int'].apply(lambda x: max(x)).max())
print(cleaned['AN_tok_int'].apply(lambda x: min(x)).min())

5458
1
